In [1]:
import os
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('../..')
os.getcwd()

'c:\\Users\\user\\Downloads\\ten_academy\\week1\\Week1-10Academy-User-Analysis'

In [3]:
from src.plots import Plot

In [11]:
df = pd.read_csv('data/tel_dump.csv')

In [5]:
plot = Plot()

In [12]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [14]:
df.isna().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [16]:
# Data Cleaning: Handling missing values for relevant columns

# Fill missing numerical values with the mean of the respective column
df['TCP DL Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'].fillna(df['TCP DL Retrans. Vol (Bytes)'].mean())
df['TCP UL Retrans. Vol (Bytes)'] = df['TCP UL Retrans. Vol (Bytes)'].fillna(df['TCP UL Retrans. Vol (Bytes)'].mean())
df['Avg RTT DL (ms)'] = df['Avg RTT DL (ms)'].fillna(df['Avg RTT DL (ms)'].mean())
df['Avg RTT UL (ms)'] = df['Avg RTT UL (ms)'].fillna(df['Avg RTT UL (ms)'].mean())

# Fill missing categorical values with the mode
df['Handset Type'] = df['Handset Type'].fillna(df['Handset Type'].mode()[0])

# Confirm that missing values have been addressed in these columns
cleaned_data_info = df[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Handset Type']].isnull().sum()

cleaned_data_info  # Output to confirm the missing values have been handled


TCP DL Retrans. Vol (Bytes)    0
TCP UL Retrans. Vol (Bytes)    0
Avg RTT DL (ms)                0
Avg RTT UL (ms)                0
Handset Type                   0
dtype: int64

In [18]:
# Task 4.1: Aggregate information per customer
# Group by customer (using 'IMSI') and calculate the required metrics

# Calculate average TCP retransmission for each customer
# Sum DL and UL retransmission values and divide by the count for averaging
avg_tcp_retransmission = df.groupby('IMSI')[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']].sum().sum(axis=1) / df.groupby('IMSI').size()

# Calculate average RTT for each customer (both downlink and uplink)
avg_rtt = df.groupby('IMSI')[['Avg RTT DL (ms)', 'Avg RTT UL (ms)']].mean().mean(axis=1)

# Calculate average throughput for each customer (both downlink and uplink)
avg_throughput = df.groupby('IMSI')[['Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']].mean().mean(axis=1)

# Identify the most frequent handset type for each customer
most_frequent_handset_type = df.groupby('IMSI')['Handset Type'].agg(lambda x: x.mode()[0])

# Combine the calculated metrics into a single dataframe
customer_experience_aggregate = pd.DataFrame({
    'IMSI': avg_tcp_retransmission.index,
    'Average TCP Retransmission': avg_tcp_retransmission.values,
    'Average RTT': avg_rtt.values,
    'Most Frequent Handset Type': most_frequent_handset_type.values,
    'Average Throughput (kbps)': avg_throughput.values
})

customer_experience_aggregate.head()  # Display the first few rows of the aggregated information per customer


,IMSI,Average TCP Retransmission,Average RTT,Most Frequent Handset Type,Average Throughput (kbps)
0,2.040471e+14,2.156957e+07,63.729294,Quectel Wireless. Quectel Ec21-E,1.00
1,2.040808e+14,2.156957e+07,63.729294,Quectel Wireless. Quectel Ec25-E,0.50
2,2.082001e+14,7.623557e+05,151.500000,Dn Electronics Danew Konnect 350,54.50
3,2.082001e+14,2.797900e+04,42.000000,Samsung Galaxy Grand (Gt-I9060X),377.00
4,2.082001e+14,2.156957e+07,46.000000,Apple iPhone 6S (A1688),40.25
